In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from scipy.special import rel_entr
import numpy as np

In [2]:
validate_processed_sample = 'data/processed/validate_processed_sample.csv'
validate_processed_target = 'data/processed/validate_processed_target.csv'
validate_processed_diff = 'data/processed/validate_processed_differential_diagnosis.csv'

In [3]:
X_val = pd.read_csv(validate_processed_sample)
y_val = pd.read_csv(validate_processed_target)

dval = xgb.DMatrix(X_val, label=y_val)

del X_val

diff_val = pd.read_csv(validate_processed_diff)
diff_val = diff_val.to_numpy()
y_val = y_val.to_numpy()

In [21]:
print("There are 49 possible pathology in the dataset.")
for top in [1, 5, 10, 20, 25]:
    predicted_classes = np.argsort(diff_val, axis=1)[:,-top:]
    correct_predictions = 0

    for i in range(len(y_val)):
        if y_val[i] in predicted_classes[i]:
            correct_predictions += 1

    print(f"{correct_predictions/y_val.shape[0] * 100}% of top {top} pathology in dataset differental diagnosis being the correct pathology")

There are 49 possible pathology in the dataset.


73.34878593863252% of top 1 pathology in dataset differental diagnosis being the correct pathology
97.2630768301522% of top 5 pathology in dataset differental diagnosis being the correct pathology
99.62400338245953% of top 10 pathology in dataset differental diagnosis being the correct pathology
99.99244986711766% of top 20 pathology in dataset differental diagnosis being the correct pathology
100.0% of top 25 pathology in dataset differental diagnosis being the correct pathology


In [31]:
bst = xgb.Booster()
model_filename = 'model/dep8_lr30_subsam7_colsam4_rnd30_sed37702_GTPA9970_DDP9967_DDR2160_F1_3550.json'
bst.load_model(model_filename)
y_pred_prob = bst.predict(dval)

print("This model is trained to predict the correct pathology.")
for top in [1, 2, 3]:
    predicted_classes = np.argsort(y_pred_prob, axis=1)[:,-top:]
    correct_predictions = 0

    for i in range(len(y_val)):
        if y_val[i] in predicted_classes[i]:
            correct_predictions += 1

    print(f"{correct_predictions/y_val.shape[0] * 100}% of top {top} pathology in differental diagnosis prediction being the correct pathology")

There are 49 possible pathology in the dataset.
99.70176975114762% of top 1 pathology in differental diagnosis prediction being the correct pathology
99.99924498671177% of top 2 pathology in differental diagnosis prediction being the correct pathology
100.0% of top 3 pathology in differental diagnosis prediction being the correct pathology
